In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
train['Class'].value_counts()

0    106597
1     10967
Name: Class, dtype: int64

In [6]:
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

In [7]:
X = train.iloc[:, 1:-1]
y = train['Class']

target = test[X.columns]

In [8]:
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

In [11]:
cb_pred = np.zeros(target.shape[0])
for i, (tr_idx, val_idx) in enumerate(skf.split(X, y)) :
    
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    cb = CatBoostClassifier(random_state = 42, max_depth = 6, learning_rate = 0.03, n_estimators = 10000, use_best_model = True, eval_metric = 'Logloss')
    cb.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], early_stopping_rounds = 1000, verbose = 2000)
    
    val_pred = cb.predict_proba(val_x)[:, 1]
    val_log_loss = log_loss(val_y, val_pred)
    print(f'{i + 1} Fold Logloss : {val_log_loss}')
    
    fold_pred = cb.predict_proba(target)[:, 1] / skf.n_splits
    cb_pred += fold_pred

0:	learn: 0.6094959	test: 0.6095766	test1: 0.6091703	best: 0.6091703 (0)	total: 9.64ms	remaining: 1m 36s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.02521166259
bestIteration = 669

Shrink model to first 670 iterations.
1 Fold Logloss : 0.0252116625872773
0:	learn: 0.6083399	test: 0.6084154	test1: 0.6082458	best: 0.6082458 (0)	total: 8.54ms	remaining: 1m 25s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.0335825041
bestIteration = 367

Shrink model to first 368 iterations.
2 Fold Logloss : 0.03358250409548504
0:	learn: 0.6090186	test: 0.6091009	test1: 0.6090848	best: 0.6090848 (0)	total: 9.7ms	remaining: 1m 36s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.02996498472
bestIteration = 831

Shrink model to first 832 iterations.
3 Fold Logloss : 0.029964984721138725
0:	learn: 0.6090170	test: 0.6091065	test1: 0.6095626	best: 0.6095626 (0)	total: 8.83ms	remaining: 1m 28s
2000:	learn: 0.0191063	test: 0.0191129	test1: 0.

In [12]:
submission = pd.read_csv('sample_submission.csv')

In [13]:
submission['Class'] = cb_pred

In [15]:
submission.to_csv('1st.csv', index = False)